In [1]:
'''
1. 特征相关性分析
    题设要求根据一天前9小时的观测数据判断第10天的PM2.5数据
    每一小时都提供了18个特征（包括PM2.5数据），九小时也就是18*9=162个特征
    然而，显然这162个特征里有很大一部分是没有太大关联的，直观上想，只有前九小时的PM2.5特征最有价值
    其次，我们也可以考虑通过前九小时的其它特征预测第10小时的指定特征，然后利用预测的17个第10小时特
    征对PM2.5特征进行二次预测，最后通过结合两次预测的结果得到预测值。
'''
print()

In [2]:
# 这里重写了预处理函数，去掉了验证集与测试集的处理，去掉了归一化
def preprocessing(raw_train_data):
    month_data = {}                             # 将初始数据按月份分开存入字典
    for month in range(12):
        sample = np.empty([18, 24 * 20])        # 每个月的数据均为：(特征数量, (24*20))
        for day in range(20):
            sample[:, day*24:(day+1)*24] = raw_train_data[18*(20*month+day):18*(20*month+day+1), :]
        month_data[month] = sample
    print('month_data[0] shape: ', month_data[0].shape)
    
    x_train = np.empty([12*471, 18*9], dtype=float) # 构建x_train，维度：(5652, 162)
    y_train = np.empty([12*471, 1], dtype=float)    # 构建y_train,维度：(5652, 1)
    for month in range(12):
        for day in range(20):
            for hour in range(24):
                if day == 19 and hour > 14:
                    continue
                x_train[month*471+day*24+hour, :] = month_data[month][:, day*24+hour : day*24+hour+9].reshape(1, -1)
                y_train[month*471+day*24+hour, 0] = month_data[month][9, day*24+hour+9]
    print('x_train shape: ', x_train.shape)
    print('y_train shape: ', y_train.shape)

    return x_train, y_train


In [3]:
# '''
# 这里我将处理好的训练集: (5652, 162)，以及标签: (5652, 1)放入csv文件
# '''

# from data_utils import *
# import pandas as pd

# data_dir = 'datasets/'
# raw_train_data, _ = get_data(data_dir)     # 1. 从数据集内读数据
# x_train, y_train = preprocessing(raw_train_data)
# input_file = pd.DataFrame(x_train)

# # 处理表头
# alist = ['AMD_TEMP', 'CH4', 'CO', 'NMHC', 'NO', 'NO2', 'NOx', '03', 'PM10', 'PM2.5',
#         'RAIN', 'RH', 'SO2', 'THC', 'WD_HR', 'WIND_DIR', 'WIND_SPEED', 'WS_HR']
# header_list = []
# for element in alist:
#     for i in range(1, 10):
#         header_list.append(element + '_' + str(i))
# input_file.columns = header_list

# input_file['label'] = y_train
# input_file.to_csv('datasets/characteristics.csv', )

In [4]:
# 检验特征相关性
import pandas as pd
import numpy as np

data = pd.read_csv('datasets/characteristics.csv', encoding='big5')
data = data.apply(lambda x:x.astype(float)).drop(columns=['Unnamed: 0'])
corr_matrix = data.corr()
label_corr = corr_matrix['label']
# print(corr_matrix)
# print(label_corr)
label_corr = label_corr.to_numpy()
label_corr = np.delete(label_corr, 162)
label_corr = label_corr.reshape(18, 9)
# print(label_corr)
corr_dic = {}

alist = ['AMD_TEMP', 'CH4', 'CO', 'NMHC', 'NO', 'NO2', 'NOx', '03', 'PM10', 'PM2.5',
        'RAIN', 'RH', 'SO2', 'THC', 'WD_HR', 'WIND_DIR', 'WIND_SPEED', 'WS_HR']
for i in range(18):
    corr_dic[alist[i]] = label_corr[i][8]

print('Characteristic Correlation: ')
count = 0
for key, value in corr_dic.items():
    print(str(key) + ': ' + str(value))
    cur_char = data.iloc[:, 9*i:9*(i+1)]
#     print(cur_char)
    file_name = str(key) + '.csv'
    i += 1
    cur_char.to_csv('datasets/' + file_name, index=False)
'''
这里检测出了几个关联性不大的特征（考虑将相关系数小于0.2的舍去），另有两个相关性非常大的特征（>0.7)
'''

Characteristic Correlation: 
AMD_TEMP: 0.0013404436338343068
CH4: 0.2614886718884517
CO: 0.3143841434545102
NMHC: 0.3266357570166916
NO: 0.07922619032631435
NO2: 0.47496992755777545
NOx: 0.41147741180872144
03: 0.37733082179609506
PM10: 0.7567723912073928
PM2.5: 0.9144170870841293
RAIN: -0.07256164379265834
RH: -0.30606952727533404
SO2: 0.4033830975544701
THC: 0.37470174628261627
WD_HR: 0.21344681917760985
WIND_DIR: 0.18693070615207474
WIND_SPEED: -0.07131088008681327
WS_HR: -0.045687287192624486


'\n这里检测出了几个关联性不大的特征（考虑将相关系数小于0.2的舍去），另有两个相关性非常大的特征（>0.7)\n'

In [5]:
# 这里我们先考虑PM2.5单个特征
data_pm25 = data.iloc[:, 81:90].to_numpy()
data_pm25_train = data_pm25[:4521, :]
data_pm25_val = data_pm25[4521:, :]
print(data_pm25_train.shape)

(4521, 9)


In [6]:
# 同时考虑PM2.5和PM10
data_pm10 = data.iloc[:, 72:90].to_numpy()
data_pm10_train = data_pm10[:4521, :]
data_pm10_val = data_pm10[4521:, :]
print(data_pm10_train.shape)

(4521, 18)


In [7]:
from data_utils import *
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

data_dir = 'datasets/'
raw_train_data, raw_test_data = get_data(data_dir)     # 1. 从数据集内读数据
x_train, y_train, x_val, y_val, x_test = data_preprocessing(raw_train_data, raw_test_data)  # 2. 数据预处理

raw_train_data shape:  (4320, 24)
raw_test_data shape:  (4320, 9)
month_data[0] shape:  (18, 480)
x_train shape:  (4521, 163)
y_train shape:  (4521, 1)
x_val shape:  (1131, 163)
y_val shape:  (1131, 1)
x_test shape:  (240, 163)


In [8]:
def train_LinearRegression():
    # LinearRegression(fit_intercept=True, normalize=False, copy_X=True, n_jobs=None)
    model = make_pipeline(PolynomialFeatures(1), LinearRegression(fit_intercept=False))
    model.fit(x_train, y_train)
    y_pred = model.predict(x_val)
    rmse = np.sqrt(np.sum(np.power(y_pred - y_val, 2)) / len(x_val)) # rmse
    print('Linear Regression(all data): rmse = ', rmse)
    return y_pred[:, 0]
alldata_pred = train_LinearRegression()

Linear Regression(all data): rmse =  5.668544173979506


In [9]:
def train_LinearRegression():
    # LinearRegression(fit_intercept=True, normalize=False, copy_X=True, n_jobs=None)
    model = make_pipeline(PolynomialFeatures(1), LinearRegression())
    model.fit(data_pm25_train, y_train)
    y_pred = model.predict(data_pm25_val)
    rmse = np.sqrt(np.sum(np.power(y_pred - y_val, 2)) / len(data_pm25_val)) # rmse
    print('Linear Regression(pm25 data): rmse = ', rmse)
    return y_pred[:, 0]
pm25_pred = train_LinearRegression()

Linear Regression(pm25 data): rmse =  5.861093589266862


In [10]:
def train_LinearRegression():
    # LinearRegression(fit_intercept=True, normalize=False, copy_X=True, n_jobs=None)
    model = make_pipeline(PolynomialFeatures(1), LinearRegression())
    model.fit(data_pm10_train, y_train)
    y_pred = model.predict(data_pm10_val)
    rmse = np.sqrt(np.sum(np.power(y_pred - y_val, 2)) / len(data_pm10_val)) # rmse
    print('Linear Regression(pm10, pm25 data): rmse = ', rmse)
    return y_pred[:, 0]
pm10_pred = train_LinearRegression()

Linear Regression(pm10, pm25 data): rmse =  5.770850986437787


In [11]:
'''
这里我将仅用pm2.5处理后的数据与之前数据进行线性组合得出新的结果，略优于baseline
'''
# print(alldata_pred)
alldata_rate = 0.70
label_val = y_val[:, 0]
y_pred = alldata_rate * alldata_pred + (1 - alldata_rate) * pm25_pred
print(y_pred)
rmse = np.sqrt(np.sum(np.power(y_pred - label_val, 2)) / len(data_pm25_val)) # rmse
print(rmse)

[14.59089086 15.96974832 19.74528744 ... 20.72917976 20.71232827
 25.19670078]
5.618701925265138


In [12]:
'''
这里我将用pm2.5和pm10处理后的数据与之前数据进行线性组合得出新的结果，略优于baseline
'''
# print(alldata_pred)
alldata_rate = 0.75
label_val = y_val[:, 0]
y_pred = alldata_rate * alldata_pred + (1 - alldata_rate) * pm10_pred
print(y_pred)
rmse = np.sqrt(np.sum(np.power(y_pred - label_val, 2)) / len(data_pm10_val)) # rmse
print(rmse)

[14.54290125 15.72055632 19.49731515 ... 20.9015675  21.08064052
 25.04434082]
5.635431641507339


In [13]:
def result_output(y_pred):
    with open('outputs/submit_version2.csv', mode='w', newline='') as submit_file:
        csv_writer = csv.writer(submit_file)
        header = ['id', 'value']
        # print(header)
        csv_writer.writerow(header)
        for i in range(240):
            row = ['id_' + str(i), y_pred[i][0]]
            csv_writer.writerow(row)
            # print(row)
    print('submission.csv file saved.')
    
def test_preprocessing(raw_test_data):
    x_test = np.empty([240, 18*9], dtype=float)
    for i in range(240):
        x_test[i, :] = raw_test_data[18*i:18*(i+1), :].reshape(1, -1)
    return x_test
    
def test_LinearRegression(x, y, x_test):
    # LinearRegression(fit_intercept=True, normalize=False, copy_X=True, n_jobs=None)
    model = make_pipeline(PolynomialFeatures(1), LinearRegression(fit_intercept=False))
    model.fit(x, y)
    y_pred = model.predict(x_test)
    return y_pred[:, 0]

In [14]:
alldata_test = test_LinearRegression(x_train, y_train, x_test)

x_test_pm25 = test_preprocessing(raw_test_data)[:, 81:90]
pm25_test = test_LinearRegression(data_pm25_train, y_train, x_test_pm25)
print(alldata_test.shape)
print(pm25_test.shape)

(240,)
(240,)


In [15]:
print(alldata_test[:10])
print(pm25_test[:10])

[ 6.88105208 18.29892736 25.23316827  7.01935355 27.33055301 22.29862428
 24.03302744 31.07633437 16.84048977 60.13113803]
[ 5.65996192 16.55222641 24.10973813  6.71575034 27.42527063 21.96049845
 24.83349239 30.54503641 17.08439492 59.75090754]


In [18]:
y_final = alldata_test * 0.7 + pm25_test * 0.3
print(y_final)
y_final = y_final[:, np.newaxis]
print(y_final.shape)
result_output(y_final)

[ 6.51472503e+00  1.77749171e+01  2.48961392e+01  6.92827258e+00
  2.73589683e+01  2.21971865e+01  2.42731669e+01  3.09169450e+01
  1.69136613e+01  6.00170689e+01  1.17177718e+01  9.82783475e+00
  6.33048452e+01  5.38765729e+01  2.24679660e+01  1.28294108e+01
  3.21386388e+01  6.60349360e+01  8.21332243e-02  1.78471530e+01
  4.16079819e+01  7.17149518e+01  8.66769103e+00  1.79659155e+01
  1.46593746e+01  3.89859811e+01  1.49042982e+01  6.81957494e+01
  7.24610245e+00  5.52522353e+01  2.35790751e+01  9.81585584e+00
  2.55598293e+00  1.92093469e+01  2.87612354e+01  3.75179374e+01
  4.30963846e+01  3.07492320e+01  4.04741393e+01  3.62660064e+01
  7.12580202e+00  4.09971424e+01  3.18557487e+01  5.06480460e+01
  1.74097794e+01  3.62230180e+01  2.43011010e+01  1.02004014e+01
  2.64894650e+01  3.24012751e+01  1.99888515e+01  7.18376181e+00
  2.36094266e+01  5.17991938e+01  1.60661672e+01  3.58207351e+01
  3.36327964e+01  2.08282718e+01  5.90081658e+01  2.26638382e+01
  1.39051572e+01  4.21974